# 矩阵分解模型

### tqdm库
- 进度条
- https://blog.csdn.net/qq_41731861/article/details/121678411

### np.bincount()
- https://blog.csdn.net/HG0724/article/details/121270521
- 统计从0到array数组中最大数字出现的个数的函数，并同样以array数组输出显示

### np.ones()
- 返回给定形状和数据类型的新数组，其中元素的值设置为1
- https://blog.csdn.net/silent1cat/article/details/119883549

### np.ceil()
- 计算大于等于该值的最小整数
- https://blog.csdn.net/devinzkx/article/details/124618434

In [88]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm  # 进度条

data = np.loadtxt('data/movielens_100k.csv', delimiter=',', dtype=int)
# 数据中的三列依次表示：用户编号，电影编号，评分
print('数据集大小：',len(data))

数据集大小： 100000


In [89]:
# 将数据全转化为序列0开始
data[:,:2] = data[:,:2] - 1

# 计算用户和电影数量
users = set()
items = set()
for i, j, k in data:
    users.add(i)
    items.add(j)
user_num = len(users)
item_num = len(items)
print(f'用户数{user_num},电影数{item_num}')

用户数943,电影数1682


In [90]:
# 划分训练集与测试集
np.random.seed(2023)
ratio = 0.8
split = int(len(data) * ratio)
np.random.shuffle(data)
train = data[:split]
test = data[split:]

# 统计训练集中每个用户和电影出现的次数，作为正则化约束的强度
user_cnt = np.bincount(train[:, 0], minlength=user_num)
item_cnt = np.bincount(train[:, 1], minlength=item_num)
print(user_cnt[:10],item_cnt[:10])

[209  52  46  20 145 177 325  49  16 143] [375  97  75 169  65  21 308 180 237  71]


In [91]:
# 用户和电影的标号要作为下标，必须保存为整数
user_train, user_test = train[:, 0], test[:, 0]
item_train, item_test = train[:, 1], test[:, 1]
y_train, y_test = train[:, 2], test[:, 2]

In [92]:
# 定义矩阵分解模型的类
class MF:
    def __init__(self, N, M, d):
        # N是用户数量，M是电影数量，d是特征维数
        # 定义模型的参数
        self.user_params = np.ones((N, d))
        self.item_params = np.ones((M, d))

    def pred(self, user_id, item_id):
        # 预测用户user_id与对电影item_id的评分
        # 获得用户偏好和电影特征
        user_param = self.user_params[user_id]
        item_param = self.item_params[item_id]
        # 返回预测的评分
        rating_pred = np.sum(user_param * item_param, axis=1)
        return rating_pred
    
    def update(self, user_grad, item_grad, lr):
        # 根据参数的梯度更新参数
        self.user_params -= lr * user_grad
        self.item_params -= lr * item_grad


In [93]:
# 定义训练函数，用SGD对MF模型的参数进行优化，MSE作为损失函数，RMSE作为评价指标
def train(model, learning_rate, lbd, max_training_step, batch_size):
    train_losses = []
    test_losses = []
    batch_num = int(np.ceil(len(user_train) / batch_size))
    with tqdm(range(max_training_step * batch_num)) as pbar:
        for epoch in range(max_training_step):
            # 随机梯度下降
            train_rmse = 0
            for i in range(batch_num):
                # 获取当前批量
                st = i * batch_size
                ed = min(len(user_train), st + batch_size)
                user_batch = user_train[st: ed]
                item_batch = item_train[st: ed]
                y_batch = y_train[st: ed]
                # 计算模型预测
                y_pred = model.pred(user_batch, item_batch)
                # 计算梯度
                P = model.user_params
                Q = model.item_params
                errs = y_batch - y_pred
                P_grad = np.zeros_like(P)
                Q_grad = np.zeros_like(Q)
                for user, item, err in zip(user_batch, item_batch, errs):
                    P_grad[user] = P_grad[user] - err * Q[item] + lbd * P[user]
                    Q_grad[item] = Q_grad[item] - err * P[user] + lbd * Q[item]
                model.update(P_grad / len(user_batch), Q_grad / len(user_batch), learning_rate)

                train_rmse += np.mean(errs ** 2)

                # 更新进度条
                pbar.set_postfix({
                    'Epoch': epoch,
                    'Train RMSE': f'{np.sqrt(train_rmse / (i + 1)):.4f}',
                    'Test RMSE': f'{test_losses[-1]:.4f}' if test_losses else None
                })
                pbar.update(1)

            # 计算测试集上的RMSE损失
            train_rmse = np.sqrt(train_rmse / len(user_train))
            train_losses.append(train_rmse) 
            y_test_pred = model.pred(user_test, item_test)
            test_rmse = np.sqrt(np.mean((y_test - y_test_pred) ** 2))
            test_losses.append(test_rmse)

    return train_losses, test_losses


In [94]:
# 设置参数
feature_num = 16  # 特征数
learning_rate = 0.1
lbd = 1e-4  # 正则化约束强度
max_training_step = 30
batch_size = 64  # 批量大小

# 建立模型
model = MF(user_num, item_num, feature_num)
# 训练
train_losses, test_losses = train(model, learning_rate, lbd, max_training_step, batch_size)

100%|██████████| 37500/37500 [01:15<00:00, 494.29it/s, Epoch=29, Train RMSE=0.9688, Test RMSE=1.0004]
